GET THE ORIGINAL DATA.  
GENERATE MATRIX FOR X,  
WITH X FROM **'op_profit_margin', 'short_leverage', 'bmr', 'epsp', 'size_d1', 'size_d2'**  
WITH Y FROM **'anual_cc_return_future'**

In [1]:
isGoogleColab = False

if(isGoogleColab):
    from google.colab import drive
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats.mstats import winsorize

In [2]:
industry_dict = {'Accommodation and Food Services':'Services',
       'Administrative and Support and Waste Management and Remediation Services':'Services',
       'Agriculture, Forestry, Fishing and Hunting':'Agriculture',
       'Arts, Entertainment, and Recreation':'Services', 
       'Construction':'Construction',
       'Educational Services':'Services', 
       'Finance and Insurance':'Finance',
       'Health Care and Social Assistance':'Services', 
       'Information':'Services',
       'Manufacturing':'Manufacturing', 
       'Mining, Quarrying, and Oil and Gas Extraction':'Mining',
       'Other Services (except Public Administration)':'Services',
       'Professional, Scientific, and Technical Services':'Services',
       'Real Estate and Rental and Leasing':'Finance', 
       'Retail Trade':'Comercial',
       'Transportation and Warehousing':'Services', 
       'Utilities':'Energy', 
       'Wholesale Trade':'Comercial'}

In [3]:
if(isGoogleColab):
    drive.mount("/content/gdrive") 
    %cd "/content/gdrive/MyDrive/Machine Learning/deliver/M1"
df = pd.read_csv('us2022q2a.csv')
df_firm = pd.read_csv('usfirms2022.csv')

In [4]:
bIsAproachNew = False

df_more = df.copy()

#put industry column
firm_list = df['firm'].unique()
firm_list = firm_list.tolist()
firm_list.remove(np.nan)
df_firm_more = df_firm.copy()
df_firm_more['industry'] = df_firm_more['Sector NAICS\nlevel 1']
df_firm_more = df_firm_more.replace({'industry': industry_dict})

firm_industry_dic = {}

if(bIsAproachNew):
  firm_industry_dic_temp = {}

  for count in range(df_firm_more['Ticker'].count()):
    firm_industry_dic_temp[df_firm_more['Ticker'][count]] = df_firm_more['industry'][count]

  #print(firm_industry_dic_temp)
else:
  for firm in firm_list:
    if (len((df_firm_more[df_firm_more['Ticker'] == firm]['industry']).values.tolist()) > 0):
      firm_industry_dic[firm] = ((df_firm_more[df_firm_more['Ticker'] == firm]['industry']).values.tolist())[0]
    else:
      firm_industry_dic[firm] = np.nan

for firm in firm_list:
    if(bIsAproachNew):
      if((len(firm) > 0) and (firm in firm_industry_dic_temp)):
        firm_industry_dic[firm] = firm_industry_dic_temp[firm]
      #else:
      #  firm_industry_dic[firm] = np.nan
    else:
      if(len(firm) > 0):
        #firm_industry_dic[firm] = ((df_firm_more[df_firm_more['Ticker'] == firm]['industry']).values.tolist())[0]
        temp_df = df_firm_more[df_firm_more['Ticker'] == firm]
        if(temp_df['industry'].count() > 0):
          firm_industry_dic[firm] = temp_df['industry'].tolist()[0]
      #else:
      #  firm_industry_dic[firm] = np.nan

df_more['industry'] = df_more['firm']
df_more = df_more.replace({'industry': firm_industry_dic})


#put country column
firm_country_dict = {}
for count in range(df_firm['Ticker'].count()):
    firm = df_firm['Ticker'][count]
    country = df_firm['Country\nof Origin'][count]
    firm_country_dict[firm] = country

df_more['country'] = df_more['firm']
df_more = df_more.replace({'country':firm_country_dict})
#drop the line which all column is empty
df_more = df_more.dropna(how='all')
#df_more

In [5]:
df_more['firm_index'] = df_more['firm']
df_more['q_index'] = df_more['q']
df_more.set_index(['firm_index','q_index'], inplace=True)
#add Date
df_more['date'] = pd.to_datetime(df_more['q'])

#Calculate bookvalue
df_more['bookvalue'] = df_more['totalassets'] - df_more['totalliabilities']

#Calculate Market Value
df_more['marketvalue'] = df_more['originalprice'] * df_more['sharesoutstanding']

#Calculate Operating profit margin
df_more['op_profit'] = df_more['revenue'] - df_more['cogs'] - df_more['sgae'] - df_more['otheropexp']
df_more['op_profit_margin'] = df_more['op_profit'] / df_more['revenue']
df_more['net_income'] = df_more['op_profit'] - df_more['incometax'] - df_more['finexp'] + df_more['extraincome']
df_more['profit_margin'] = df_more['net_income'] / df_more['revenue']

#Calculate CC Return
df_more['anual_cc_return'] = np.log(df_more['adjprice']) - np.log(df_more['adjprice'].shift(4))
df_more['anual_cc_return_future'] = np.log(df_more['adjprice'].shift(-1)) - np.log(df_more['adjprice'].shift(3))


#Earning Per Share
df_more['eps'] = df_more['net_income'] / df_more['sharesoutstanding']

#Calculate the short term leverage

#Calculate the ratio of bookvalue and market value


df_more = df_more.dropna(subset=['adjprice'])

In [6]:
industry_list = df_more['industry'].unique()
industry_list = industry_list.tolist()
del industry_list[-1]
del industry_list[-1]

In [7]:
df_more.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
df_more_clean = df_more.dropna(subset=['op_profit_margin', 'anual_cc_return', 'eps'])
winsorize(df_more_clean['op_profit_margin'], limits=[0.01,0.01],  inplace=True)
winsorize(df_more_clean['anual_cc_return'], limits=[0.01,0.01],inplace=True)
winsorize(df_more_clean['eps'], limits=[0.01,0.01], inplace=True)

masked_array(data=[ 0.33715018,  0.21003221, -0.47542827, ...,
                    0.28896335, -1.12921415, -0.44195264],
             mask=False,
       fill_value=1e+20)

In [9]:
df_more['epsp'] = df_more['eps'] / df_more['originalprice']

In [10]:
df_finance = df_more[df_more['industry'] == 'Finance'].copy()
df_finance['short_leverage'] = df_finance['shortdebt'] / df_finance['totalassets']
df_finance['bmr'] = df_finance['bookvalue'] / df_finance['marketvalue']

df_finance.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
df_finance_train = df_finance.copy()
df_finance_train = df_finance_train.dropna(subset=['op_profit_margin', 'short_leverage', 'bmr', 'epsp', 'anual_cc_return_future'])

In [12]:
df_finance_train['op_profit_margin'] = winsorize(df_finance_train['op_profit_margin'], limits=[0.01,0.01])
df_finance_train['short_leverage'] = winsorize(df_finance_train['short_leverage'], limits=[0.01,0.01])
df_finance_train['bmr'] = winsorize(df_finance_train['bmr'], limits=[0.01,0.01])
df_finance_train['epsp'] = winsorize(df_finance_train['epsp'], limits=[0.01,0.01])

In [13]:
df_finance_train['size_d1'] = 0
df_finance_train['size_d2'] = 0

In [14]:
df_finance_train_copy = df_finance_train.copy()
df_finance_train_date_size = df_finance_train_copy.sort_values(['date', 'marketvalue']).groupby('date')
df_finance_train_date_size[['marketvalue', 'date', 'q']].head()
#len(df_finance_train_date_size['date'].unique())
#df_finance_train_date_size['date'].head()

count = 0
for date in df_finance_train_copy['date'].unique():
    df_date = df_finance_train_copy.loc[df_finance_train_copy['date'] == date]
    
    size_total = df_date['marketvalue'].count()
    size_per = size_total // 3
    
    df_date_size_sort = df_date.sort_values('marketvalue')
    #df_date_size_sort.reset_index()
    #if(count == 0):
    #    print(df_date_size_sort[['size_d1', 'marketvalue']].head())
    #df_date_size_sort['size_d1'].iloc[size_per:(2 * size_per - 1)] = 1
    #df_date_size_sort['size_d2'].iloc[(2 * size_per) :] = 1

    for firm in df_date_size_sort['firm'].iloc[size_per:(2 * size_per - 1)]:
        df_finance_train_copy.loc[(df_finance_train_copy['firm'] == firm) & (df_finance_train_copy['date'] == date), 'size_d1'] = 1
        #if(count == 0):
        #    print(df_finance_train_copy[(df_finance_train_copy['firm'] == firm) & (df_finance_train_copy['date'] == date)].index.to_list()[0])

    for firm in df_date_size_sort['firm'].iloc[(2 * size_per) :]:
        df_finance_train_copy.loc[(df_finance_train_copy['firm'] == firm) & (df_finance_train_copy['date'] == date), 'size_d2'] = 1


    count += 1

#print(count)
#df_finance_train_copy.isna().sum()
#df_finance_train_copy[['date', 'marketvalue', 'size_d1', 'size_d2']].head(40)
#print(df_finance_train_copy['size_d1'].unique())
#print(df_finance_train_copy['size_d2'].unique())

1 Perform an exploratory analysis of the variables:(You have to use matrix algebra and check results with Python functions.)

1.1 Calculates variance and covariance matrix,   

as well as correlation matrix of the independent and dependent variables

Interpret the correlation matrix.

VarCov(X) = (1/(N - 1)) ( X'.X - (1/N)*(X'.1).(X'.1) )

## Explain what variance, covariance, and correlation are.

**Variance**, is a measure of variability from the mean  

**Covariance**, is a measure of relationship between the variability of 2 variables  
It is the Average of the Product Deviations from the mean of each variable  
We can not understand the magnitude of Covariance   

**Correlation**, is a relationship between the variability of 2 variables  
Correlation is standarized, value between -1 and 1

In [15]:
X = df_finance_train_copy[['op_profit_margin', 'short_leverage', 'bmr', 'epsp', 'size_d1', 'size_d2']]
y = df_finance_train_copy['anual_cc_return_future']

In [16]:
X_Mat = X.to_numpy()
X_Mat.shape


(7346, 6)

Variance = 1/N Sum((Xi - X_Avg)^2)

X_Avg = X . (ONE x (1/N))  

In [17]:
X_Avg = np.dot(np.ones((1, X_Mat.shape[0])), X_Mat) / (X_Mat.shape[0])
X_Avg.shape

(1, 6)

In [18]:
X.mean()

op_profit_margin    0.133567
short_leverage      0.043115
bmr                 0.581360
epsp                0.005564
size_d1             0.316499
size_d2             0.342499
dtype: float64

In [19]:
X_Avg

array([[0.13356652, 0.04311546, 0.5813598 , 0.00556411, 0.31649877,
        0.34249932]])

X_Diff = X - (ONE . X_Avg)

In [20]:
X_Mat

array([[-0.82498624,  0.        ,  0.99599506,  0.00439069,  1.        ,
         0.        ],
       [ 0.00810064,  0.        ,  1.04992656,  0.00438099,  1.        ,
         0.        ],
       [-0.86865851,  0.        ,  1.00591696, -0.01491826,  1.        ,
         0.        ],
       ...,
       [-0.11597549,  0.04811628,  2.81872096, -0.03901772,  0.        ,
         0.        ],
       [-0.87021487,  0.02947186,  3.51525056, -0.29726164,  0.        ,
         0.        ],
       [-0.34640448,  0.02865596,  2.44275162, -0.11957765,  0.        ,
         0.        ]])

In [21]:
X_Diff = X_Mat - (np.dot(np.ones((X_Mat.shape[0], 1)), X_Avg))
X_Diff

array([[-9.58552763e-01, -4.31154554e-02,  4.14635255e-01,
        -1.17341507e-03,  6.83501225e-01, -3.42499319e-01],
       [-1.25465878e-01, -4.31154554e-02,  4.68566759e-01,
        -1.18311652e-03,  6.83501225e-01, -3.42499319e-01],
       [-1.00222503e+00, -4.31154554e-02,  4.24557156e-01,
        -2.04823647e-02,  6.83501225e-01, -3.42499319e-01],
       ...,
       [-2.49542013e-01,  5.00082239e-03,  2.23736116e+00,
        -4.45818245e-02, -3.16498775e-01, -3.42499319e-01],
       [-1.00378139e+00, -1.36435956e-02,  2.93389076e+00,
        -3.02825744e-01, -3.16498775e-01, -3.42499319e-01],
       [-4.79971003e-01, -1.44594967e-02,  1.86139181e+00,
        -1.25141759e-01, -3.16498775e-01, -3.42499319e-01]])

X_Diff2 = X_Diff x X_Diff

In [22]:
X_Diff2 = X_Diff * X_Diff
X_Diff2.shape

(7346, 6)

Sum_X_Diff2 = ONE . X_Diff2
Variance = Sum_X_Diff2 / N

In [23]:
Sum_X_Diff2 = np.dot(np.ones((1, X_Diff2.shape[0])), X_Diff2)
Variance = Sum_X_Diff2 / X_Diff2.shape[0]
Variance

array([[0.25313993, 0.01015089, 0.45027479, 0.00226093, 0.2163273 ,
        0.22519354]])

In [24]:
X.var()

op_profit_margin    0.253174
short_leverage      0.010152
bmr                 0.450336
epsp                0.002261
size_d1             0.216357
size_d2             0.225224
dtype: float64

In [26]:
X.cov()

,op_profit_margin,short_leverage,bmr,epsp,size_d1,size_d2
op_profit_margin,0.253174,-0.000484,-0.032765,0.008987,0.005183,0.034739
short_leverage,-0.000484,0.010152,0.005007,-0.000050,0.003663,-0.003454
bmr,-0.032765,0.005007,0.450336,-0.004712,-0.018215,-0.054351
epsp,0.008987,-0.000050,-0.004712,0.002261,0.001375,0.002286
size_d1,0.005183,0.003663,-0.018215,0.001375,0.216357,-0.108415
size_d2,0.034739,-0.003454,-0.054351,0.002286,-0.108415,0.225224


In [27]:
X.corr()

,op_profit_margin,short_leverage,bmr,epsp,size_d1,size_d2
op_profit_margin,1.000000,-0.009547,-0.097035,0.375586,0.022144,0.145478
short_leverage,-0.009547,1.000000,0.074047,-0.010406,0.078162,-0.072241
bmr,-0.097035,0.074047,1.000000,-0.147657,-0.058354,-0.170659
epsp,0.375586,-0.010406,-0.147657,1.000000,0.062155,0.101287
size_d1,0.022144,0.078162,-0.058354,0.062155,1.000000,-0.491132
size_d2,0.145478,-0.072241,-0.170659,0.101287,-0.491132,1.000000


Use the Variant Covariant Matrix

In [25]:
X_Mat

array([[-0.82498624,  0.        ,  0.99599506,  0.00439069,  1.        ,
         0.        ],
       [ 0.00810064,  0.        ,  1.04992656,  0.00438099,  1.        ,
         0.        ],
       [-0.86865851,  0.        ,  1.00591696, -0.01491826,  1.        ,
         0.        ],
       ...,
       [-0.11597549,  0.04811628,  2.81872096, -0.03901772,  0.        ,
         0.        ],
       [-0.87021487,  0.02947186,  3.51525056, -0.29726164,  0.        ,
         0.        ],
       [-0.34640448,  0.02865596,  2.44275162, -0.11957765,  0.        ,
         0.        ]])

Run statistical tests to detect outliers and leverage points.  
You have to use matrix algebra for the proofs and clearly explain how the proofs work.  
You can use Python functions to check results.

B=(X'.X)^(-1) . X' . Y